In [1]:
#Import the data wrangling library pandas 
import pandas as pd

#We will also need the numpy library in this code 
import numpy as np

In the state of NY dataset, sometimes School ID don't import well and may need to be checked in the original CSV (there's a weird bug). 

It also means that it is useful to work on a copy of the dataset rather than the original in case problems arise. 

In [2]:
#Import data from New York
#This is the file with all the exemptions - you might have a cleaner and more useful version of it 
NY_exempt = pd.read_csv('C:/Data_various/New_York/NY_exemptions.csv')

#Get the data for the year 2012 
#In school year, this actually means 2012-2013 
#In this dataset, the column is actually called 'Report Period'
#If you have already added a 'Year' column to the dataset, you would use that here instead
NY_exempt_2012 = NY_exempt[NY_exempt['Report Period'] == '2012-2013']

#Add an actual year column
#You might have already done this - in that case disregard 
NY_exempt_2012['Year'] = 2012

#Enrollment data in NY is separated by public/private, and in separate years for each file
#Start by importing the data for public schools in year 2012
Enrollment_public_2012 = pd.read_csv('C:/Data_various/New_York/NY_Public_2012.csv')
#In that data, select the only two columns we care about: the ID of the schools and the number of children enrolled
Enrollment_public_2012 = Enrollment_public_2012[['STATE LOCATION ID', 'Enrollment']]
#Because we will want to be consistent with private schools, give the school ID a standard name by renaming its column
Enrollment_public_2012.rename(columns = {'STATE LOCATION ID' : 'School ID'}, inplace = True)


#Repeat the process for private schools
Enrollment_private_2012 = pd.read_csv('C:/Data_various/New_York/NY_Private_2012.csv') 
#BEDS code is actually the school ID for private schools in NY
Enrollment_private_2012 = Enrollment_private_2012[['BEDS Code', 'Enrollment']]
#Rename BEDS code to School ID 
Enrollment_private_2012.rename(columns = {'BEDS Code' : 'School ID'}, inplace = True)

#Now we have two datasets of enrollment for public and private schools, but we need both
#Since they have columns with the same name, we append them to have it as one
#The resulting dataframe is called Enrollment_2012
Enrollment_2012 = Enrollment_public_2012.append(Enrollment_private_2012)

#We can now merge the exemptions dataset without enrollment and the enrollment dataset
#They both a school ID column which we can use for that purpose 
#how = 'inner' means that we are only keeping entries that are in both dataset - thus excluding school for which we don't have enrollment
#It is worth checking with df.shape that after merging, several thousand schools remain. 
#Sometimes School ID don't import well and may need to be checked in the original CSV (there's a weird bug)
NY_exempt_enrolled_2012 = pd.merge(NY_exempt_2012, Enrollment_2012, how = 'inner', on = [ 'School ID'])

#Now we have exemption percentages and enrollment, we can calculate the number of exemptions 
#To do so we simply take the percentage of medical/religious exemptions and divide by 100 to obtain a proportion 
#We then multiply the proportion by the number enrolled. 
#Because these are individuals and the calculation is imprecise, we round the number to only obtain integers (i.e. 10 exemptions, rather 10.05 exemptions)
NY_exempt_enrolled_2012['PME_total'] = np.round((NY_exempt_enrolled_2012['Percent Medical Exemptions'] / 100) * NY_exempt_enrolled_2012['Enrollment'])
NY_exempt_enrolled_2012['PBE_total'] = np.round((NY_exempt_enrolled_2012['Percent Religious Exemptions'] / 100) * NY_exempt_enrolled_2012['Enrollment'])

#Finally, because all the other files have a column 'Total_Enrolled' rather than 'Enrollment', we can rename that column
#inplace = True means that the dataset is permanently assigned the new column name
NY_exempt_enrolled_2012.rename({'Enrollment' : 'Total_Enrolled'}, inplace = True)

#Last thing to do is to export the dataset with number of enrolled student
NY_exempt_enrolled_2012.to_csv('C:/Data_various/New_York/Enrolled_2012.csv', index = False)

C:\Users\romai\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
